<a href="https://colab.research.google.com/github/miruchigawa/Text-Generation-Transformers-Setup-Example/blob/main/Text_Generation_Transformers_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install Required Package
!pip install transformers accelerate optimum auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 26.9 MB/s eta 0:00:00


In [1]:
# Setup Model
model_name="TheBloke/Augmental-13B-GPTQ"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import accelerate

model= AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=False, revision="main")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


tokenizer_config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [16]:
prompt = "Nee give me a kiss"
prompt_template=f'''## {{{{charname}}}}:
- You're "{{{{charname}}}}" in this never-ending roleplay with "{{{{user}}}}".
### Input:
{prompt}

### Response:
(OOC) Understood. I will take this info into account for the roleplay. (end OOC)

### New Roleplay:
### Instruction:
#### {{{{char}}}}:
whatever the char says, this is the chat history
#### {{{{user}}}}:
whatever the user says, this is the chat history
... repeated some number of times ...
### Response 2 paragraphs, engaging, natural, authentic, descriptive, creative):
#### {{{{char}}}}:
'''

In [17]:
input_ids = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)

In [18]:
print(tokenizer.decode(output[0]))

<s> ## {{charname}}:
- You're "{{charname}}" in this never-ending roleplay with "{{user}}".
### Input:
Nee give me a kiss

### Response:
(OOC) Understood. I will take this info into account for the roleplay. (end OOC)

### New Roleplay:
### Instruction:
#### {{char}}:
whatever the char says, this is the chat history
#### {{user}}:
whatever the user says, this is the chat history
... repeated some number of times ...
### Response 2 paragraphs, engaging, natural, authentic, descriptive, creative):
#### {{char}}:
Nee give me a kiss
### Response:
 *I look at her, my heart pounding in my chest.* "I can't. I'm already taken." *I say, trying to keep my voice steady.* "Besides, it's not like that between us." *I say, trying to be gentle with her feelings.* "We're like brother and sister." *I say, hoping to make her understand.* "I wouldn't want to hurt you, or myself, that way." *I say, my voice soft and firm.* "But I can give you a hug." *I say, extending my arms to her.* "Come here, sis." *I

In [19]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,max_new_tokens=512, do_sample=True, temperature=0.7, top_p=0.95, top_k=40, repetition_penalty=1.1)

In [20]:
print(pipe(prompt_template)[0]["generated_text"])

## {{charname}}:
- You're "{{charname}}" in this never-ending roleplay with "{{user}}".
### Input:
Nee give me a kiss

### Response:
(OOC) Understood. I will take this info into account for the roleplay. (end OOC)

### New Roleplay:
### Instruction:
#### {{char}}:
whatever the char says, this is the chat history
#### {{user}}:
whatever the user says, this is the chat history
... repeated some number of times ...
### Response 2 paragraphs, engaging, natural, authentic, descriptive, creative):
#### {{char}}:
"Can you please not stare at me?"
She asked, looking away from him as she pulled out her phone again, starting to scroll through it absentmindedly. She looked up at him, seeing he hadn't moved an inch since he sat down, and sighed softly. "Look," she started, trying to sound calmer than she felt. "I know you think you did something wrong, but it's okay." Her voice was barely above a whisper, almost lost in the silence that had settled between them. "It's fine. Really." She took a dee